In [1]:
from comet_ml import Experiment
from comet_ml.api import API, APIExperiment
import pandas as pd
from neuralprophet import NeuralProphet, set_log_level, save, set_random_seed
import pickle
import os
from IPython.display import clear_output



# api = API(os.environ.get('COMET_ML_API_KEY'))
api = API('IbXp3DT5kVkFPKwnzhg3mgUTL')

set_log_level("CRITICAL")
set_random_seed(221110)

target_column = 'Total Vertical TBS'

df = pd.read_csv(
    'https://raw.githubusercontent.com/drdevinhopkins/hourly-report/main/data/since-2020.csv')
df.ds = pd.to_datetime(df.ds)
df = df.drop(['Date', 'Time'], axis=1)
df = df.sort_values(by='ds', ascending=True)
df.rename(columns={target_column: 'y', 'Adm. requests cum': "Adm requests cum",
          'Pts.waiting for admission CUM': 'Pts waiting for admission CUM'}, inplace=True)
df.dropna(inplace=True)
regressors = df.columns.tolist()
regressors.remove('y')
regressors.remove('ds')

for param in [0]:

    experiment = Experiment(
    # api_key=os.environ.get('COMET_ML_API_KEY'),
    api_key='IbXp3DT5kVkFPKwnzhg3mgUTL',
    project_name="verticalTBS-tweaking-hidden-layers",
    workspace="drdevinhopkins",
)

    params = {
        # growth='off',
        'yearly_seasonality': False,
        'weekly_seasonality': True,
        'daily_seasonality': True,
        'n_lags': 17,
        'n_forecasts': 12,
        # 'changepoints_range': 0.95,
        # 'n_changepoints': 50,
        'quantiles': [0.2, 0.5, 0.8],
        # 'num_hidden_layers':0
        # d_hidden=36,
        # learning_rate=0.005,
    }

    m = NeuralProphet(**params)
    m = m.add_lagged_regressor(names=regressors)
    m = m.add_country_holidays("CA")
    df_train, df_test = m.split_df(df, valid_p=0.05)

    metrics = m.fit(df_train,
                    freq='H',
                    validation_df=df_test
                    # progress='plot'
                    )
    print(metrics.tail(1))

    experiment.log_parameters(params)
    experiment.log_metrics(metrics.tail(1).iloc[0].to_dict())

    experiment.end()

    clear_output(wait=True)


COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/drdevinhopkins/verticaltbs-tweaking-hidden-layers/a05129c419a047a8bfd767c782a6cbe1

WARNING - (py.warnings._showwarnmsg) - /home/codespace/.local/lib/python3.10/site-packages/neuralprophet/df_utils.py:1036: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, "ds"] = pd.to_datetime(df.loc[:, "ds"])

WARNING - (py.warnings._showwarnmsg) - /home/codespace/.local/lib/python3.10/site-packages/neuralprophet/df_utils.py:1036: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of 

  0%|          | 0/159 [00:00<?, ?it/s]

WARNING - (py.warnings._showwarnmsg) - /home/codespace/.local/lib/python3.10/site-packages/torch/nn/modules/loss.py:928: UserWarning: Using a target size (torch.Size([64, 12, 1])) that is different to the input size (torch.Size([64, 12, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.smooth_l1_loss(input, target, reduction=self.reduction, beta=self.beta)

WARNING - (py.warnings._showwarnmsg) - /home/codespace/.local/lib/python3.10/site-packages/torch/nn/modules/loss.py:928: UserWarning: Using a target size (torch.Size([318, 12, 1])) that is different to the input size (torch.Size([318, 12, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.smooth_l1_loss(input, target, reduction=self.reduction, beta=self.beta)



  0%|          | 0/159 [00:00<?, ?it/s]

COMET INFO: ---------------------------                                                                                                                                              
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/drdevinhopkins/verticaltbs-tweaking-hidden-layers/a05129c419a047a8bfd767c782a6cbe1
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     Loss             : 0.02137163043329235
COMET INFO:     MAE              : 3.7003215699775045
COMET INFO:     MAE_val          : 5.3952185397897825
COMET INFO:     RMSE             : 4.840660240511485
COMET INFO:     RMSE_val         : 6.933195044493923
COMET INFO:     RegLoss          : 0.0
COMET INFO:     SmoothL1Loss     : 0.01499717800100049
COMET INFO:     SmoothL1Loss_val : 0.030716458512418096
COMET INFO:     loss [3088]      : (0.01610429398715496, 284.852569580

    SmoothL1Loss       MAE     RMSE      Loss  RegLoss  SmoothL1Loss_val  \
81      0.014997  3.700322  4.84066  0.021372      0.0          0.030716   

     MAE_val  RMSE_val  
81  5.395219  6.933195  


COMET ERROR: Error sending a notification, make sure you have opted-in for notifications
COMET INFO: Uploading metrics, params, and assets to Comet before program termination (may take several seconds)
COMET INFO: The Python SDK has 3600 seconds to finish before aborting...


In [16]:
import pandas as pd
from neuralprophet import NeuralProphet, set_log_level, save, set_random_seed

set_log_level("CRITICAL")
set_random_seed(221110)

target_column = 'Total Vertical TBS'

df = pd.read_csv(
    'https://raw.githubusercontent.com/drdevinhopkins/hourly-report/main/data/since-2020.csv')
df.ds = pd.to_datetime(df.ds)
df = df.drop(['Date', 'Time'], axis=1)
df = df.sort_values(by='ds', ascending=True)
df.rename(columns={target_column: 'y', 'Adm. requests cum': "Adm requests cum",
          'Pts.waiting for admission CUM': 'Pts waiting for admission CUM'}, inplace=True)
df.dropna(inplace=True)
regressors = df.columns.tolist()
regressors.remove('y')
regressors.remove('ds')

METRICS = ["SmoothL1Loss", "MAE", "RMSE"]

params = {
    # growth='off',
    'yearly_seasonality': False,
    'weekly_seasonality': True,
    'daily_seasonality': True,
    'n_lags': 17,
    'n_forecasts': 12,
    # 'changepoints_range': 0.95,
    # 'n_changepoints': 50,
    # 'quantiles': [0.2, 0.5, 0.8],
    # 'num_hidden_layers':0
    # d_hidden=36,
    # learning_rate=0.005,
}

folds = NeuralProphet(**params).add_lagged_regressor(names=regressors).add_country_holidays("CA").crossvalidation_split_df(df, freq="H", k=5, fold_pct=0.20, fold_overlap_pct=0.5)

metrics_train = pd.DataFrame(columns=METRICS)
metrics_test = pd.DataFrame(columns=METRICS)

for df_train, df_test in folds:
    m = NeuralProphet(**params).add_lagged_regressor(names=regressors).add_country_holidays("CA")
    train = m.fit(df=df_train, freq='H')
    test = m.test(df=df_test)
    metrics_train = metrics_train.append(train[METRICS].iloc[-1])
    metrics_test = metrics_test.append(test[METRICS].iloc[-1])

metrics_test.describe().loc[["mean", "std", "min", "max"]]


WARNING - (py.warnings._showwarnmsg) - /home/codespace/.local/lib/python3.10/site-packages/neuralprophet/df_utils.py:1036: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, "ds"] = pd.to_datetime(df.loc[:, "ds"])

WARNING - (py.warnings._showwarnmsg) - /home/codespace/.local/lib/python3.10/site-packages/neuralprophet/df_utils.py:1036: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, "ds"] = pd.to_datetime(df.loc[:, "ds"])

WARNING - (py.warnings._showwarnmsg) - /home/codespace/.local/lib/python3.10/site-packages/neuralprophet/df_utils.py

ValueError: Encountered variable with singular value in training set. Please remove variable.

In [13]:
regressors

['Stretcher Pts hrly',
 'Stretcher Pts cum',
 'Ambulatory Pts hrly',
 'Ambulatory Pts cum',
 'Total Inflow hrly',
 'Total Inflow cum',
 'Ambulances hrly',
 'Ambulances cum',
 'FLS hrly',
 'Adm requests cum',
 'Admissions cum',
 'Pts waiting for admission CUM',
 'Total Stretcher pts',
 'Triage hallway pts',
 'Triage hallway pts TBS',
 'Re-Oriented Nurse cum',
 'Re-Oriented MD QTrack D/C',
 'Re-Oriented MD QTrack NotD/C',
 'Resus Pts',
 'Totalpts in PODs except Psych',
 'Green Pts',
 'Green Pts TBS',
 'Yellow PTS',
 'Yellow Pts TBS',
 'Orancge Pts except psych',
 'Orange Pts TBS',
 'Consults > 2h in PODS except IM',
 'Consult for IM >4h in PODS',
 'Plain films reqs > 2 h in PODs',
 'CTs reqs > 2 h in PODs',
 'Post POD (Family room)',
 'Stretcher Pts in Vertical',
 'Stretcher Pts TBS in Vertical',
 'Stretcher Pts in Vertical on Lazyboy',
 'Vertical Pts Waiting for Results',
 'Ambulatory Pts in Vertical',
 'Ambulatory Pts TBS in Vertical',
 'QTrack Patients TBS',
 'GARAGE patient TBS',
 'C